# 2018 KBO Mock Rookie Draft

## Formulars
* WAR = RAR / 10
* RAR = RAA + 20 (PA / 600)
* RAA = wRAA + wSB + UBR + UZR + Position Constant
* wRAA = ((wOBA - league wOBA)/wOBAscale) * PA

타자<br/>
** wOBA(weight on Base Average) = (0.735 * (NIBB+HBP) + 0.90 * 1B + 1.24 * 2B + 1.56 * 3B + 1.95 * HR) / PA **

투수<br/>
** FIP = { ( 13 x HR ) + ( 3 x BB ) – ( 2 x K ) } / 이닝 + cFIP **

NIBB : Not Intentional base on balls<br/>
IBB : Intentional base on balls<br/>
PA : Plate appearance

** References<br/> **
https://en.wikipedia.org/wiki/WOBA<br/>
http://ko.yagongso.wikidok.net/wp-d/59e0e5f411f7fa6a3ea5545a/View



## 1. 타자

In [24]:
import pandas as pd
import numpy as np

#Import data
hitter = pd.read_csv('2017hitter.csv')
hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 619 entries, 0 to 618
Data columns (total 17 columns):
순위     619 non-null int64
이름     619 non-null object
경기     619 non-null int64
타율     619 non-null float64
타석     619 non-null int64
타수     619 non-null int64
안타     619 non-null int64
2루타    619 non-null int64
3루타    619 non-null int64
홈런     619 non-null int64
타점     619 non-null int64
득점     619 non-null int64
사사구    619 non-null int64
삼진     619 non-null int64
출루율    619 non-null float64
장타율    619 non-null float64
도루     619 non-null int64
dtypes: float64(3), int64(13), object(1)
memory usage: 82.3+ KB


### Data Cleaning for Hitter 

In [25]:
#check missing value
hitter.isnull().sum()

순위     0
이름     0
경기     0
타율     0
타석     0
타수     0
안타     0
2루타    0
3루타    0
홈런     0
타점     0
득점     0
사사구    0
삼진     0
출루율    0
장타율    0
도루     0
dtype: int64

In [26]:
hitter.head()

,순위,이름,경기,타율,타석,타수,안타,2루타,3루타,홈런,타점,득점,사사구,삼진,출루율,장타율,도루
0,1,장재우(광천고),1,1.000,4,4,4,0,0,0,0,2,0,0,1.000,1.000,1
1,2,문정재(광천고),1,0.500,4,2,1,0,1,0,1,1,0,0,0.500,1.500,0
2,3,배지환(경북고),27,0.474,120,95,45,3,5,1,17,31,20,10,0.556,0.642,30
3,4,천현재(부경고),15,0.463,53,41,19,6,3,0,13,9,11,2,0.566,0.756,2
4,5,김민석(부경고),15,0.462,36,26,12,1,0,1,9,7,9,3,0.583,0.615,1


규정타석을 채우지 못한 타자들을 거를 필요가 있다 <br/>

2017년 기준
* 전반기 주말리그 6.5경기
* 후반기 주말리그 6.5경기
* 황금사자기
* 청룡선수권
* 대통령배
* 봉황대기
* 전국체전 

전국체전을 제외한 대회를 각각 1.5경기를 평균으로 해서 총 19경기를 소속팀 평균 경기수로 계산했다

규정타석 : 소속팀 경기수 x 0.8 x 3 = 46타석 <br/>
http://www.hsbaseball.kr/board/bbs_cmu_read.php?idxno=972&menu_idxno=11&page=31&search_item=&search_word=&category=

In [40]:
hitter = hitter[hitter['타석'] >= 46]
hitter.shape

(553, 19)

In [41]:
#단타 지표 추가
hitter['단타'] = hitter.apply(lambda x: x[6] - (x[7] + x[8] +x[9]), axis=1)
col_names = ['순위','이름','경기','타율','타석','타수','안타','단타','2루타','3루타','홈런','타점','득점','사사구','삼진','출루율','장타율','도루']
hitter = hitter.reindex(columns=col_names)
hitter.head()

,순위,이름,경기,타율,타석,타수,안타,단타,2루타,3루타,홈런,타점,득점,사사구,삼진,출루율,장타율,도루
16,17,박영완(대구고),14,0.400,59,35,14,0,5,2,0,13,14,20,6,0.596,0.657,2
5,6,안영환(신흥고),16,0.452,46,31,14,0,3,1,0,6,8,12,1,0.605,0.613,2
3,4,천현재(부경고),15,0.463,53,41,19,0,6,3,0,13,9,11,2,0.566,0.756,2
23,24,김민기(덕수고),30,0.393,124,89,35,1,8,1,1,13,27,31,10,0.545,0.539,13
25,26,추재현(신일고),22,0.390,99,77,30,3,6,3,3,25,23,22,4,0.525,0.662,4


In [42]:
#타자 종합지표 wOBA도입
hitter['wOBA'] = hitter.apply(lambda x: (0.735*x[13] + 0.9*x[7] + 1.24*x[8] + 1.56*x[9] + 1.95*x[10])/x[5], axis=1)
hitter = hitter.sort_values(by='wOBA',ascending=0)
hitter.head(10)

,순위,이름,경기,타율,타석,타수,안타,단타,2루타,3루타,홈런,타점,득점,사사구,삼진,출루율,장타율,도루,wOBA
16,17,박영완(대구고),14,0.400,59,35,14,0,5,2,0,13,14,20,6,0.596,0.657,2,0.686286
235,236,박경식(효천고),17,0.280,69,50,14,0,7,2,0,8,10,19,12,0.478,0.500,3,0.515300
86,87,장준환(유신고),21,0.340,68,50,17,1,8,1,1,10,16,15,5,0.485,0.600,3,0.507100
3,4,천현재(부경고),15,0.463,53,41,19,0,6,3,0,13,9,11,2,0.566,0.756,2,0.492805
406,407,조대현(유신고),21,0.220,91,59,13,2,3,0,2,21,13,26,12,0.438,0.373,2,0.483559
25,26,추재현(신일고),22,0.390,99,77,30,3,6,3,3,25,23,22,4,0.525,0.662,4,0.478442
206,207,이성원(장안고),17,0.291,74,55,16,2,5,0,2,18,9,18,15,0.459,0.491,0,0.456909
5,6,안영환(신흥고),16,0.452,46,31,14,0,3,1,0,6,8,12,1,0.605,0.613,2,0.454839
122,123,고민성(신흥고),16,0.320,64,50,16,0,6,3,0,7,10,14,9,0.469,0.560,3,0.448200
71,72,김태원(야탑고),23,0.350,98,80,28,2,10,3,2,22,22,17,9,0.459,0.625,7,0.440937


## 2. 투수

In [31]:
pitcher = pd.read_csv('2017pitcher.csv')
pitcher.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 20 columns):
순위        233 non-null int64
이름(팀명)    233 non-null object
경기        233 non-null int64
승         233 non-null int64
패         233 non-null int64
세이브       233 non-null int64
이닝        233 non-null float64
타자        233 non-null int64
타수        233 non-null int64
피안타       233 non-null int64
피홈        233 non-null int64
희생        233 non-null int64
4구        233 non-null int64
사구        233 non-null int64
삼진        233 non-null int64
실점        233 non-null int64
자책        233 non-null int64
투구        233 non-null int64
S         233 non-null int64
방어율       233 non-null float64
dtypes: float64(2), int64(17), object(1)
memory usage: 36.5+ KB


### Data Cleaning for Pitcher 

In [32]:
pitcher.isnull().sum()

순위        0
이름(팀명)    0
경기        0
승         0
패         0
세이브       0
이닝        0
타자        0
타수        0
피안타       0
피홈        0
희생        0
4구        0
사구        0
삼진        0
실점        0
자책        0
투구        0
S         0
방어율       0
dtype: int64

In [33]:
pitcher.head()

,순위,이름(팀명),경기,승,패,세이브,이닝,타자,타수,피안타,피홈,희생,4구,사구,삼진,실점,자책,투구,S,방어율
0,1,김정우(동산고),24,5,0,0,28.0,107,95,15,0,2,5,3,29,5,3,422,0,0.965
1,2,김영준(선린인고),18,5,1,0,65.0,258,219,44,0,12,15,8,55,19,7,987,0,0.970
2,3,안승민(원주고),14,0,1,0,17.2,73,63,11,0,1,5,4,8,6,2,282,0,1.020
3,4,남가현(배명고),23,2,1,0,26.0,103,85,16,0,6,9,1,18,5,3,381,0,1.040
4,5,정철원(안산공고),23,9,0,0,85.0,334,290,59,0,7,30,7,82,17,10,1230,0,1.059


규정이닝 : 소속팀 경기수(19) x 0.8 = 15.2타석 <br/>

In [39]:
pitcher = pitcher[pitcher['이닝'] >= 15.2]
pitcher.shape

(226, 20)

In [54]:
#FIP = { ( 13 x HR ) + ( 3 x BB ) – ( 2 x K ) } / 이닝 + cFIP
pitcher['FIP'] = pitcher.apply(lambda x: ( 13*x[10] + 3*x[12] - 2*x[14])/ x[6] + 3.16, axis=1)
pitcher = pitcher.sort_values(by='FIP',ascending=True)
pitcher.head()

,순위,이름(팀명),경기,승,패,세이브,이닝,타자,타수,피안타,...,희생,4구,사구,삼진,실점,자책,투구,S,방어율,FIP
49,49,곽빈(배명고),23,2,2,0,28.1,112,92,16,...,8,6,6,41,9,7,431,0,2.225,0.882420
28,29,전용주(안산공고),23,3,1,0,40.1,157,135,26,...,4,9,5,58,10,8,610,0,1.786,1.264738
13,14,양창섭(덕수고),30,7,2,0,50.1,198,178,39,...,9,7,2,56,10,8,692,0,1.431,1.343633
65,66,강백호(서울고),31,3,2,0,31.2,143,116,27,...,5,14,4,49,15,9,560,0,2.559,1.365128
117,118,윤강찬(김해고),13,4,4,0,66.1,276,247,64,...,8,6,15,73,36,26,994,0,3.529,1.420212


,순위,이름(팀명),경기,승,패,세이브,이닝,타자,타수,피안타,...,희생,4구,사구,삼진,실점,자책,투구,S,방어율,FIP
65,66,강백호(서울고),31,3,2,0,31.2,143,116,27,...,5,14,4,49,15,9,560,0,2.559,1.365128
86,87,주승우(서울고),31,7,1,0,77.0,303,267,64,...,13,21,2,64,29,26,1143,0,3.041,2.822338
13,14,양창섭(덕수고),30,7,2,0,50.1,198,178,39,...,9,7,2,56,10,8,692,0,1.431,1.343633
6,7,최민준(경남고),30,11,1,0,81.0,315,279,42,...,4,21,11,94,15,10,1148,0,1.111,1.616790
41,42,서준원(경남고),30,7,2,0,85.1,345,301,62,...,14,14,10,84,26,20,1223,0,2.110,2.137673
123,124,김동찬(덕수고),30,1,3,0,36.2,164,125,33,...,4,27,8,26,23,15,642,0,3.689,4.320221
7,8,박동수(덕수고),30,7,0,0,60.2,240,209,47,...,6,18,5,63,13,8,952,0,1.188,1.963987
9,10,김의준(동성고),29,8,1,0,91.2,367,320,60,...,9,31,5,88,17,13,1407,0,1.278,2.249912
8,9,김기훈(동성고),29,5,2,0,58.0,231,185,25,...,7,28,9,77,12,8,982,0,1.242,2.625517
10,11,신기인(동성고),29,5,1,0,35.0,138,119,23,...,3,9,7,33,9,5,503,0,1.287,2.045714
